# Check Results
This notebook I will use to save each cost type, each agency correction, and the final numbers to a .csv. These .csv's we will use for visualizing results and checking code. The code here will also be the basis of my test code for refactoring.

## Set up Environment

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import os
import sys
import numpy as np
from sodapy import Socrata

In [3]:
sys.path.insert(0, os.path.abspath("../"))

In [4]:
from Initialize_Agencies import get_agencies
yr = list(range(2016,2020))
agencies = get_agencies(yr)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/Users/alexanderweinstein/Documents/Harris/Summer2020/Carceral_Budgeting/Exploratory/Cost_Type_Code/Pensions/Statewide_Pensions.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

In [5]:
final_results = pd.read_csv("Final_by_Agency_Type.csv", index_col = 0)
final_results.rename(columns = {x:int(x) for x in final_results.columns[3:]}, inplace=True)

# Check

In [6]:
index_attr_dict = {" Payroll Costs": "payroll_by_year",
                   " Non-Payroll Operating Costs": "non_payroll_operating_expenditures_by_year",
                   " Pension Costs": "pensions",
                   " Fringe Benefit Costs": "fringe",
                   " Capital Costs": "capital_expenditures_by_year"}
passed = True
for _, agency in agencies.items():
    for cti, cta in index_attr_dict.items():
        true = final_results.loc[agency.alias + cti , yr].astype(float).values
        current = agency.correction_function(agency.__dict__[cta]).astype(float).values 
        if not np.allclose(a=true, b=current, rtol=1e-10):
            print(cti + " for "+ agency.alias+ " not equal. Correct number is " + str(true) + \
                  " . New number is " + str(current))
            print()
            passed = False

if passed: 
    print("all passed!!") 
else:
    print("Some mismatches")

all passed!!


Note that this doesn't check local pensions for trial court

# Check get_final_costs
Wrote new get_final_costs method, see if it works here

In [7]:
cost_types = [" Payroll Costs", " Non-Payroll Operating Costs", 
              " Pension Costs", 
              " Fringe Benefit Costs", " Capital Costs"]
for _, agency in agencies.items():
    current_costs = agency.get_final_costs(True, True)
    for pos, cost_type in enumerate(cost_types):
        true = final_results.loc[agency.alias + cost_type , yr].astype(float).values
        current = current_costs[pos+1]
        if not np.allclose(a=true, b=current.values, rtol=1e-10):
            print(cost_type + " for "+ agency.alias+ " not equal. Correct number is " + str(true) + \
                  " . New number is " + str(current))
            print()
            passed = False

if passed: 
    print("all passed!!") 
else:
    print("Some mismatches")
        

percent criminal correction is  2016   0.343
2017   0.335
2018   0.358
2019   0.380
dtype: float64
 Pension Costs for trial_court not equal. Correct number is [3568017.52345819 3772546.40782839 4348113.45907107 4638839.0233857 ] . New number is 2016   5117059.819
2017   5396153.173
2018   6177160.111
2019   6626866.819
dtype: float64

Some mismatches


TRC was supposed to get different number lol

In [8]:
TRC = agencies["trial_court"]

In [9]:
total, payroll, non_payroll_operating, final_pensions, final_fringe, final_capital  = TRC.get_final_costs(True, True)

percent criminal correction is  2016   0.343
2017   0.335
2018   0.358
2019   0.380
dtype: float64


In [10]:
final_pensions

2016   5117059.819
2017   5396153.173
2018   6177160.111
2019   6626866.819
dtype: float64

In [11]:
TRC.correction_function(TRC.pensions)

2016   3568017.523
2017   3772546.408
2018   4348113.459
2019   4638839.023
dtype: float64

In [12]:
final_results.loc[TRC.alias + " Pension Costs" , yr].astype(float).values

array([3568017.52345819, 3772546.40782839, 4348113.45907107,
       4638839.0233857 ])